In [115]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import RFE
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.pipeline import make_pipeline
from xgboost import XGBClassifier

Using matplotlib backend: QtAgg


In [116]:
# importation de donnees 
df = pd.read_csv("database_TT_Churn.csv")
df 

df.head()

,id_client,genre,age,marie,num_tel,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,...,nb_appel_nuit,cout_appel_nuit,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type
0,382-4657,femme,37,yes,98505453,128.0,265.1,110.0,45.07,197.4,...,91.0,11.01,10.0,3.0,2.70,yes,25,1.0,False,Hayya
1,371-7191,homme,46,no,97321658,107.0,161.6,123.0,27.47,195.5,...,103.0,11.45,13.7,3.0,3.70,yes,26,1.0,False,PRE - 1=11
2,358-1921,homme,50,no,98653270,137.0,243.4,114.0,41.38,121.2,...,104.0,7.32,12.2,5.0,3.29,no,0,0.0,False,PRE - 900 bonus
3,375-9999,homme,78,yes,96303256,84.0,299.4,71.0,50.90,61.9,...,89.0,8.86,6.6,7.0,1.78,no,0,2.0,False,PRE - AHLA
4,330-6626,femme,75,yes,96412387,75.0,166.7,113.0,28.34,148.3,...,121.0,8.41,10.1,3.0,2.73,no,0,3.0,False,PRE - Binetna


In [117]:
df.shape

(5000, 23)

In [118]:
df.isna().sum()

id_client              0
genre                  0
age                    0
marie                  0
num_tel                0
nb_jours_abonne       10
duree_appel_jour       0
nb_appel_jour          7
cout_appel_jour        0
duree_appel_soiree     0
nb_appel_soiree        6
cout_appel_soiree      0
duree_appel_nuit       0
nb_appel_nuit         11
cout_appel_nuit        0
duree_appel_inter      0
nb_appel_inter        12
cout_appel_inter       0
active_msg_vocaux      8
nb_msg_vocaux          0
nb_reclamation        12
churn                  4
offer_type             0
dtype: int64

In [119]:
# information sur l'ensemble de donnees
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_client           5000 non-null   object 
 1   genre               5000 non-null   object 
 2   age                 5000 non-null   int64  
 3   marie               5000 non-null   object 
 4   num_tel             5000 non-null   int64  
 5   nb_jours_abonne     4990 non-null   float64
 6   duree_appel_jour    5000 non-null   float64
 7   nb_appel_jour       4993 non-null   float64
 8   cout_appel_jour     5000 non-null   float64
 9   duree_appel_soiree  5000 non-null   float64
 10  nb_appel_soiree     4994 non-null   float64
 11  cout_appel_soiree   5000 non-null   float64
 12  duree_appel_nuit    5000 non-null   float64
 13  nb_appel_nuit       4989 non-null   float64
 14  cout_appel_nuit     5000 non-null   float64
 15  duree_appel_inter   5000 non-null   float64
 16  nb_app

In [8]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
age,5000.0,4.650680e+01,16.743513,19.0,3.200000e+01,46.00,60.00,80.00
num_tel,5000.0,9.844445e+07,371630.462063,90032100.0,9.850545e+07,98505453.00,98505453.00,99652014.00
nb_jours_abonne,4990.0,1.002707e+02,39.704130,1.0,7.300000e+01,100.00,127.00,243.00
duree_appel_jour,5000.0,1.802889e+02,53.894699,0.0,1.437000e+02,180.10,216.20,351.50
nb_appel_jour,4993.0,1.000190e+02,19.828023,0.0,8.700000e+01,100.00,113.00,165.00
cout_appel_jour,5000.0,3.064967e+01,9.162069,0.0,2.443000e+01,30.62,36.75,59.76
duree_appel_soiree,5000.0,2.006366e+02,50.551309,0.0,1.663750e+02,201.00,234.10,363.70
nb_appel_soiree,4994.0,1.002147e+02,19.820534,0.0,8.700000e+01,100.00,114.00,170.00
cout_appel_soiree,5000.0,1.705432e+01,4.296843,0.0,1.414000e+01,17.09,19.90,30.91
duree_appel_nuit,5000.0,2.003916e+02,50.527789,0.0,1.669000e+02,200.40,234.70,395.00


In [120]:
# nombre de valeurs uniques par colonne 
df.nunique()

id_client             5000
genre                    2
age                     62
marie                    2
num_tel                 33
nb_jours_abonne        218
duree_appel_jour      1961
nb_appel_jour          123
cout_appel_jour       1961
duree_appel_soiree    1879
nb_appel_soiree        126
cout_appel_soiree     1659
duree_appel_nuit      1853
nb_appel_nuit          131
cout_appel_nuit       1028
duree_appel_inter      170
nb_appel_inter          21
cout_appel_inter       170
active_msg_vocaux        2
nb_msg_vocaux           48
nb_reclamation          10
churn                    2
offer_type              40
dtype: int64

In [121]:
#ANALYSE UNIVARIEE

In [122]:
# varaiables catregorielles
categorical_columns = ['genre', 'marie', 'offer_type','active_msg_vocaux', 'churn']
categorical_columns

['genre', 'marie', 'offer_type', 'active_msg_vocaux', 'churn']

In [125]:
# creation d une fonction de construction de histogrammes a barres et de maniere interactive
def plot_distribution(column):
    
    if column == 'churn':
        # Plot donut chart for 'churn' column
        plt.figure(figsize=(8, 6))
        df[column].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['#ff9999','#66b3ff'], startangle=90, wedgeprops=dict(width=0.3), labels=None)
        centre_circle = plt.Circle((0,0),0.70,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        plt.axis('equal')  
        plt.title('Churn Distribution')
        plt.legend(labels=df[column].value_counts().index, loc='best')
        plt.show()
    else:
        # Plot bar chart for other columns
        plt.figure(figsize=(8, 6))
        ax = sns.countplot(x=column, data=df, palette='coolwarm')
        plt.ylabel('Nombre de Clients')
        plt.title('Distribution de ' + column)
        plt.xticks(rotation=45)
        
        # Add percentage labels to the bars for columns other than 'offer_type'
        if column != 'offer_type':
            total = len(df[column])
            for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_height() / total)
                x = p.get_x() + p.get_width() / 2 - 0.05
                y = p.get_height()
                ax.annotate(percentage, (x, y), ha='center', va='bottom')
        
        plt.show()

# Now you can use interact
interact(plot_distribution, column=categorical_columns)


interactive(children=(Dropdown(description='column', options=('genre', 'marie', 'offer_type', 'active_msg_voca…

<function __main__.plot_distribution(column)>

In [13]:
# varaiables quantitatives
numerical_columns = ['age','nb_jours_abonne','duree_appel_jour','nb_appel_jour','cout_appel_jour','duree_appel_soiree','nb_appel_soiree',
                    'cout_appel_soiree','duree_appel_nuit','nb_appel_nuit','cout_appel_nuit','duree_appel_inter','nb_appel_inter',
                    'cout_appel_inter','nb_msg_vocaux','nb_reclamation']
numerical_columns

['age',
 'nb_jours_abonne',
 'duree_appel_jour',
 'nb_appel_jour',
 'cout_appel_jour',
 'duree_appel_soiree',
 'nb_appel_soiree',
 'cout_appel_soiree',
 'duree_appel_nuit',
 'nb_appel_nuit',
 'cout_appel_nuit',
 'duree_appel_inter',
 'nb_appel_inter',
 'cout_appel_inter',
 'nb_msg_vocaux',
 'nb_reclamation']

In [126]:
# Distribution d'age
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', data=df,  color='#FFB6C1',order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Distribution d'âge")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend( loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

c:\Users\chino\anaconda3\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [127]:
#Distribution de Nombre d'Appel pour chaque type d'appel

appel_jour = df['nb_appel_jour'].sum()
appel_soiree = df['nb_appel_soiree'].sum()
appel_nuit = df['nb_appel_nuit'].sum()
appel_inter = df['duree_appel_inter'].sum()

# Labels for each type of call
labels = ['Appels de Jour', 'Appels de Soirée', 'Appels de Nuit', 'Appels Internationaux']

x = np.arange(len(labels))
width = 0.35

# Creating the bar plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Distribution du Nombre d'Appel pour chaque type d'appel")
bars = ax.bar(x, [appel_jour, appel_soiree, appel_nuit, appel_inter], width, color='#F4CCCC')

# Adding labels, title, and legend
ax.set_xlabel("Types d'appel")
ax.set_ylabel('Nombre des appels')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45)
ax.legend(loc='upper right')

# Displaying the plot
plt.tight_layout()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [128]:
#Distribution de la Duration des Appels pour chaque type d'appel


# Select the columns containing the durations for each time period
jour_duration = df['duree_appel_jour']
soiree_duration = df['duree_appel_soiree']
nuit_duration = df['duree_appel_nuit']
inter_duration = df['duree_appel_inter']
# Define pastel colors
colors = [(1, 0.75, 0.8), (0.75, 0.85, 1), (0.85, 1, 0.85),(0.7, 0.6, 0.8)]  # Pastel shades of pink, blue, and green

# Create subplots for each time period
plt.figure(figsize=(10, 6))

# Plot the distribution of durations for each time period without bars
sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
sns.distplot(soiree_duration, hist=False, color=colors[1], label='Soiree')
sns.distplot(nuit_duration, hist=False, color=colors[2], label='Nuit')
sns.distplot(inter_duration, hist=False, color=colors[3], label='Internationale')

# Add labels and title
plt.xlabel('Duration')
plt.ylabel('Density')
plt.title('Distribution of Call Durations by Time Period')
plt.legend()

# Show the plot
plt.show()


C:\Users\chino\AppData\Local\Temp\ipykernel_19404\591958290.py:16: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
c:\Users\chino\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\chino\AppData\Local\Temp\ipykernel_19404\591958290.py:17: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displ

In [17]:
#Distribution du Cout des Appels pour chaque type d'appel
count_data = pd.DataFrame({
    'Time_Period': ['Jour', 'Soiree', 'Nuit','Internationaux'],
    'Count': [df['cout_appel_jour'].sum(), df['cout_appel_soiree'].sum(), df['cout_appel_nuit'].sum(),df['cout_appel_inter'].sum()]
})

# Define pastel colors
colors = [(0.6, 0.4, 0.8)]  # Shades of pink, blue, and green

# Create grouped bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)

# Add labels and title
plt.xlabel("Types d'appels")
plt.ylabel("Nombre d'Appels")
plt.title("Distribution du Cout des Appels pour chaque type d'appel")

# Show the plot
plt.show()


In [129]:
#  Distribution du nombre des réclamations
plt.figure(figsize=(8,8))
sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])
total = len(df['nb_reclamation'])
plt.xlabel("Nombre de réclamations ")
plt.ylabel("Nombre de clients")
plt.title("Distribution du nombre des réclamations")


Text(0.5, 1.0, 'Distribution du nombre des réclamations')

In [19]:
# creation d'une fonction de constsruction de boites a moustaches et de maniere interactive 
def box_plot(column):
    sns.boxplot(y =df[column])
    plt.title('Boxplot of '+str(column))
    return plt.show()
#interact
interact(box_plot, column= numerical_columns)

interactive(children=(Dropdown(description='column', options=('age', 'nb_jours_abonne', 'duree_appel_jour', 'n…

<function __main__.box_plot(column)>

In [20]:
#ANALYSE BIVARIEE

In [21]:
#genre vs churn

In [133]:
#nb_reclamation vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='nb_reclamation', hue='churn', data=df)
plt.title('Répartition du churn selon nombre de réclamations')
plt.xlabel('Nombre de réclamation')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.show()

In [23]:
sns.boxplot(x='churn', y='nb_reclamation', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Nombre de réclamation')
plt.show()

In [130]:
#etat_civile vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='marie', hue='churn', data=df)
plt.title("Répartition du churn selon l'état civile")
plt.xlabel('marié')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [131]:
#age vs churn

# Define age intervals
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Répartition du churn selon l'age")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


c:\Users\chino\anaconda3\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\chino\anaconda3\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)


In [26]:
sns.boxplot(x='churn', y='age', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Age ')
plt.show()

In [27]:
#nb_jours_abonne vs churn

# Define subscription days intervals
abonnement_intervals = [(0, 50), (50, 100), (100, 150), (150, 200), (200, 250)]

# Create a new column in the DataFrame to represent subscription days intervals
df['subscription_interval'] = pd.cut(df['nb_jours_abonne'], bins=[interval[0] for interval in abonnement_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])

# Plot the bar chart using subscription_interval
plt.figure(figsize=(10, 6))
sns.countplot(x='subscription_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])
plt.title("Répartition du churn selon la période d'abonnement")
plt.xlabel("Période d'abonnement en jours")
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


c:\Users\chino\anaconda3\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\chino\anaconda3\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)


In [28]:
sns.boxplot(x='churn', y='nb_jours_abonne', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Période d'abonnement en jours")
plt.show()

In [132]:
#active_msg_vocaux vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='active_msg_vocaux', hue='churn', data=df)
plt.title(" Répartition du churn selon l’activation des messages vocaux")
plt.xlabel('Active messages vocaux')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [30]:
# Calculate the percentage distribution of churn based on the activation of voicemail messages
voicemail_churn_distribution = df.groupby('active_msg_vocaux')['churn'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()

# Pivot the table to make it easier to read
voicemail_churn_pivot = voicemail_churn_distribution.pivot(index='active_msg_vocaux', columns='churn', values='percentage')

# Rename the columns for better clarity
voicemail_churn_pivot.columns = ['Not Churned (%)', 'Churned (%)']

# Display the table
print(voicemail_churn_pivot)


                   Not Churned (%)  Churned (%)
active_msg_vocaux                              
 no                      83.556040    16.443960
 yes                     92.278577     7.721423


In [31]:
#nb_msg_vocaux vs churn 
sns.boxplot(x='churn', y='nb_msg_vocaux', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nombre de message vocaux")
plt.title(" Répartition du churn selon nombre de message vocaux")
plt.show()

In [32]:
#cout_appel_jour vs churn
sns.boxplot(x='churn', y='cout_appel_jour', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Jour")
plt.title(" Répartition du churn selon le cout d'appel par jour")
plt.show()

In [33]:
#cout_appel_soiree vs churn
sns.boxplot(x='churn', y='cout_appel_soiree', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Soirée")
plt.title("Répartition du churn selon le cout d'appel par soirée")
plt.show()

In [34]:
#cout_appel_nuit vs churn
sns.boxplot(x='churn', y='cout_appel_nuit', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nuit")
plt.title("Répartition du churn selon le cout d'appel par nuit")
plt.show()

In [35]:
#cout_appel_intern vs churn
sns.boxplot(x='churn', y='cout_appel_inter', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("International")
plt.title("Répartition du churn selon le cout d'appels internationaux")
plt.show()

In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

colors = ['#225ead', '#e1cbd7']
sns.lmplot(x='duree_appel_jour', y='duree_appel_soiree', data=df, hue='churn', palette=colors)
plt.title("Relation entre la durée d'appel de jour et de soirée pour les Clients churn et non churn")
plt.show()


In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing the data
colors = ['#225ead', '#e1cbd7']
sns.lmplot(x='duree_appel_jour', y='duree_appel_nuit', data=df, hue='churn', palette='pastel')
plt.title("Relation entre la durée d'appel de jour et de nuit pour les Clients churn et non churn")
plt.show()


In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing the data
colors = ['#9b59b6', '#2ecc71']  
sns.lmplot(x='duree_appel_soiree', y='duree_appel_nuit', data=df, hue='churn', palette=colors)
plt.title("Relation entre la durée d'appel de nuit et de soirée pour les Clients churn et non churn")
plt.show()


In [39]:
df.isna().sum()

id_client                 0
genre                     0
age                       0
marie                     0
num_tel                   0
nb_jours_abonne          10
duree_appel_jour          0
nb_appel_jour             7
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           6
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit            11
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter           12
cout_appel_inter          0
active_msg_vocaux         8
nb_msg_vocaux             0
nb_reclamation           12
churn                     4
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [40]:
print(df.duplicated().sum())

0


In [41]:
#Prétrqitement des données

In [42]:
for x in df.columns :
    print(x, len(df[x].unique()))

id_client 5000
genre 2
age 62
marie 2
num_tel  33
nb_jours_abonne 219
duree_appel_jour 1961
nb_appel_jour 124
cout_appel_jour 1961
duree_appel_soiree 1879
nb_appel_soiree 127
cout_appel_soiree 1659
duree_appel_nuit 1853
nb_appel_nuit 132
cout_appel_nuit 1028
duree_appel_inter 170
nb_appel_inter 22
cout_appel_inter 170
active_msg_vocaux 3
nb_msg_vocaux 48
nb_reclamation 11
churn 3
offer_type 40
age_interval 4
subscription_interval 6


In [43]:
df = df.drop(df.columns[[0, 4]], axis=1)
df.head()


,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,...,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type,age_interval,subscription_interval
0,femme,37,yes,128.0,265.1,110.0,45.07,197.4,99.0,16.78,...,10.0,3.0,2.70,yes,25,1.0,False,Hayya,30-50,100-150
1,homme,46,no,107.0,161.6,123.0,27.47,195.5,103.0,16.62,...,13.7,3.0,3.70,yes,26,1.0,False,PRE - 1=11,30-50,100-150
2,homme,50,no,137.0,243.4,114.0,41.38,121.2,110.0,10.30,...,12.2,5.0,3.29,no,0,0.0,False,PRE - 900 bonus,30-50,100-150
3,homme,78,yes,84.0,299.4,71.0,50.90,61.9,88.0,5.26,...,6.6,7.0,1.78,no,0,2.0,False,PRE - AHLA,70-120,50-100
4,femme,75,yes,75.0,166.7,113.0,28.34,148.3,122.0,12.61,...,10.1,3.0,2.73,no,0,3.0,False,PRE - Binetna,70-120,50-100


In [44]:
#Vérifier les valeurs nuls 
df.isnull().sum()

genre                     0
age                       0
marie                     0
nb_jours_abonne          10
duree_appel_jour          0
nb_appel_jour             7
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           6
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit            11
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter           12
cout_appel_inter          0
active_msg_vocaux         8
nb_msg_vocaux             0
nb_reclamation           12
churn                     4
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [45]:
#Encodage des donnees 

In [46]:
df['nb_jours_abonne'].value_counts()

nb_jours_abonne
90.0     65
87.0     59
105.0    57
93.0     57
112.0    56
         ..
215.0     1
238.0     1
216.0     1
208.0     1
233.0     1
Name: count, Length: 218, dtype: int64

In [47]:
mean_nb_jours_abonne = int(df['nb_jours_abonne'].mean())
print(mean_nb_jours_abonne)

100


In [48]:
df['nb_appel_jour'].value_counts()

nb_appel_jour
105.0    117
102.0    113
95.0     108
97.0     104
94.0     103
        ... 
36.0       1
163.0      1
35.0       1
165.0      1
46.0       1
Name: count, Length: 123, dtype: int64

In [49]:
mean_nb_appel_jour = int(df['nb_appel_jour'].mean())
print(mean_nb_appel_jour)

100


In [50]:
df['nb_appel_soiree'].value_counts()

nb_appel_soiree
105.0    115
97.0     110
91.0     110
94.0     106
103.0    106
        ... 
156.0      1
36.0       1
12.0       1
45.0       1
38.0       1
Name: count, Length: 126, dtype: int64

In [51]:
mean_nb_appel_soiree = int(df['nb_appel_soiree'].mean())
print(mean_nb_appel_soiree)

100


In [52]:
df['nb_appel_nuit'].value_counts()

nb_appel_nuit
105.0    121
102.0    109
100.0    108
104.0    106
99.0     104
        ... 
33.0       1
166.0      1
44.0       1
175.0      1
0.0        1
Name: count, Length: 131, dtype: int64

In [53]:
mean_nb_appel_nuit= int(df['nb_appel_nuit'].mean())
print(mean_nb_appel_nuit)

99


In [54]:
df['nb_jours_abonne'].fillna(mean_nb_jours_abonne, inplace=True)
df['nb_appel_jour'].fillna(mean_nb_appel_jour, inplace=True)
df['nb_appel_soiree'].fillna(mean_nb_appel_soiree, inplace=True)
df['nb_appel_nuit'].fillna(mean_nb_appel_nuit, inplace=True)

In [55]:
df['nb_reclamation'].value_counts()

nb_reclamation
1.0    1781
2.0    1126
0.0    1020
3.0     663
4.0     252
5.0      95
6.0      34
7.0      13
9.0       2
8.0       2
Name: count, dtype: int64

In [56]:
mode_nb_reclamation = int(df['nb_reclamation'].mode()[0])
print(mode_nb_reclamation)

1


In [57]:
df['nb_reclamation'].fillna(mode_nb_reclamation, inplace=True)

In [58]:
df['nb_appel_inter'].value_counts()

nb_appel_inter
3.0     991
4.0     950
2.0     742
5.0     705
6.0     493
7.0     305
1.0     265
8.0     172
9.0     148
10.0     76
11.0     44
0.0      24
12.0     23
13.0     19
15.0      9
16.0      7
14.0      6
18.0      4
19.0      2
17.0      2
20.0      1
Name: count, dtype: int64

In [59]:
mean_nb_appel_inter = int(df[(df['nb_appel_inter'] >= 2) & (df['nb_appel_inter'] <= 7)]['nb_appel_inter'].mean())
print(mean_nb_appel_inter)

4


In [60]:

df['nb_appel_inter'].fillna(mean_nb_appel_inter, inplace=True)

In [61]:
df['active_msg_vocaux'].value_counts()

active_msg_vocaux
 no     3671
 yes    1321
Name: count, dtype: int64

In [62]:
df['active_msg_vocaux'].fillna(method ="ffill", inplace = True)

C:\Users\chino\AppData\Local\Temp\ipykernel_19404\37469121.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['active_msg_vocaux'].fillna(method ="ffill", inplace = True)


In [63]:
df['churn'].value_counts()

churn
 False    4289
 True      707
Name: count, dtype: int64

In [64]:
df.dropna(subset=['churn'], inplace=True)

In [65]:
df.isnull().sum()

genre                     0
age                       0
marie                     0
nb_jours_abonne           0
duree_appel_jour          0
nb_appel_jour             0
cout_appel_jour           0
duree_appel_soiree        0
nb_appel_soiree           0
cout_appel_soiree         0
duree_appel_nuit          0
nb_appel_nuit             0
cout_appel_nuit           0
duree_appel_inter         0
nb_appel_inter            0
cout_appel_inter          0
active_msg_vocaux         0
nb_msg_vocaux             0
nb_reclamation            0
churn                     0
offer_type                0
age_interval              0
subscription_interval    10
dtype: int64

In [66]:
#Vérifier s'il y a des doublons 
print(df.duplicated().sum())

0


In [67]:
df.dtypes

genre                      object
age                         int64
marie                      object
nb_jours_abonne           float64
duree_appel_jour          float64
nb_appel_jour             float64
cout_appel_jour           float64
duree_appel_soiree        float64
nb_appel_soiree           float64
cout_appel_soiree         float64
duree_appel_nuit          float64
nb_appel_nuit             float64
cout_appel_nuit           float64
duree_appel_inter         float64
nb_appel_inter            float64
cout_appel_inter          float64
active_msg_vocaux          object
nb_msg_vocaux               int64
nb_reclamation            float64
churn                      object
offer_type                 object
age_interval             category
subscription_interval    category
dtype: object

In [68]:
label_encoder = LabelEncoder()
df['genre'] = label_encoder.fit_transform(df['genre'])
df['marie'] = label_encoder.fit_transform(df['marie'])
df['active_msg_vocaux'] = label_encoder.fit_transform(df['active_msg_vocaux'])
df['churn'] = label_encoder.fit_transform(df['churn'])

In [69]:
# Calculer la moyenne de 'encoded_churn' pour chaque type d'offre
target_mean_df = df.groupby('offer_type')['churn'].mean().reset_index()
offer_type_mapping = dict(zip(target_mean_df['offer_type'], target_mean_df['churn']))

df['offer_type'] = df['offer_type'].map(offer_type_mapping)


In [70]:
# Save the DataFrame to a CSV file
df.to_csv('encoded_dataset.csv', index=False)  # Set index=False to avoid saving the index
df


,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,...,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type,age_interval,subscription_interval
0,0,37,1,128.0,265.1,110.0,45.07,197.4,99.0,16.78,...,10.0,3.0,2.70,1,25,1.0,0,0.166667,30-50,100-150
1,1,46,0,107.0,161.6,123.0,27.47,195.5,103.0,16.62,...,13.7,3.0,3.70,1,26,1.0,0,0.139344,30-50,100-150
2,1,50,0,137.0,243.4,114.0,41.38,121.2,110.0,10.30,...,12.2,5.0,3.29,0,0,0.0,0,0.122137,30-50,100-150
3,1,78,1,84.0,299.4,71.0,50.90,61.9,88.0,5.26,...,6.6,7.0,1.78,0,0,2.0,0,0.144860,70-120,50-100
4,0,75,1,75.0,166.7,113.0,28.34,148.3,122.0,12.61,...,10.1,3.0,2.73,0,0,3.0,0,0.116732,70-120,50-100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,65,0,50.0,235.7,127.0,40.07,223.0,126.0,18.96,...,9.9,5.0,2.67,1,40,2.0,0,0.122137,50-70,0-50
4996,1,46,1,152.0,184.2,90.0,31.31,256.8,73.0,21.83,...,14.7,2.0,3.97,0,0,3.0,1,0.144860,30-50,150-200
4997,0,77,0,61.0,140.6,89.0,23.90,172.8,128.0,14.69,...,13.6,4.0,3.67,0,0,1.0,0,0.116732,70-120,50-100
4998,1,62,0,109.0,188.8,67.0,32.10,171.7,92.0,14.59,...,8.5,6.0,2.30,0,0,0.0,0,0.135952,50-70,100-150


In [71]:
# Select only numerical columns for correlation calculation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plot the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='Pastel2', fmt=".2f")

# Add title and adjust layout
plt.title('Matrice de Corrélation')
plt.tight_layout()

# Show the plot
plt.show()

In [72]:
# Select only numeric columns for correlation calculation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
corr = numeric_df.corr()[['churn']]

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plot the heatmap
sns.heatmap(corr, cmap='crest', annot=True, center=0.0)

# Add title and adjust layout
plt.title('Corrélation entre "churn" et les autres variables', fontsize=16)
plt.tight_layout()

# Show the plot
plt.show()


In [81]:
#MODELISATION

In [82]:
data = pd.read_csv('encoded_dataset.csv')
data

,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,...,nb_appel_nuit,cout_appel_nuit,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type
0,0,37,1,128,265.1,110,45.07,197.4,99,16.78,...,91,11.01,10.0,3,2.70,1,25,1,0,0.166667
1,1,46,0,107,161.6,123,27.47,195.5,103,16.62,...,103,11.45,13.7,3,3.70,1,26,1,0,0.139344
2,1,50,0,137,243.4,114,41.38,121.2,110,10.30,...,104,7.32,12.2,5,3.29,0,0,0,0,0.122137
3,1,78,1,84,299.4,71,50.90,61.9,88,5.26,...,89,8.86,6.6,7,1.78,0,0,2,0,0.144860
4,0,75,1,75,166.7,113,28.34,148.3,122,12.61,...,121,8.41,10.1,3,2.73,0,0,3,0,0.116732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,0,65,0,50,235.7,127,40.07,223.0,126,18.96,...,116,13.39,9.9,5,2.67,1,40,2,0,0.122137
4992,1,46,1,152,184.2,90,31.31,256.8,73,21.83,...,113,9.61,14.7,2,3.97,0,0,3,1,0.144860
4993,0,77,0,61,140.6,89,23.90,172.8,128,14.69,...,97,9.56,13.6,4,3.67,0,0,1,0,0.116732
4994,1,62,0,109,188.8,67,32.10,171.7,92,14.59,...,89,10.10,8.5,6,2.30,0,0,0,0,0.135952


In [83]:
data.isna().sum()

genre                 0
age                   0
marie                 0
nb_jours_abonne       0
duree_appel_jour      0
nb_appel_jour         0
cout_appel_jour       0
duree_appel_soiree    0
nb_appel_soiree       0
cout_appel_soiree     0
duree_appel_nuit      0
nb_appel_nuit         0
cout_appel_nuit       0
duree_appel_inter     0
nb_appel_inter        0
cout_appel_inter      0
active_msg_vocaux     0
nb_msg_vocaux         0
nb_reclamation        0
churn                 0
offer_type            0
dtype: int64

In [84]:
#Extraction des variables prédictives et cible
x=data.drop(["churn"], axis =1)
y=data["churn"]
x

,genre,age,marie,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,nb_appel_soiree,cout_appel_soiree,duree_appel_nuit,nb_appel_nuit,cout_appel_nuit,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,offer_type
0,0,37,1,128,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,25,1,0.166667
1,1,46,0,107,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,26,1,0.139344
2,1,50,0,137,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0.122137
3,1,78,1,84,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,0,0,2,0.144860
4,0,75,1,75,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,0,0,3,0.116732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,0,65,0,50,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,1,40,2,0.122137
4992,1,46,1,152,184.2,90,31.31,256.8,73,21.83,213.6,113,9.61,14.7,2,3.97,0,0,3,0.144860
4993,0,77,0,61,140.6,89,23.90,172.8,128,14.69,212.4,97,9.56,13.6,4,3.67,0,0,1,0.116732
4994,1,62,0,109,188.8,67,32.10,171.7,92,14.59,224.4,89,10.10,8.5,6,2.30,0,0,0,0.135952


In [85]:
np.random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=15, stratify=y)
print("Train set shape:", x_train.shape, y_train.shape)
print("Test set shape:", x_test.shape, y_test.shape)

Train set shape: (3996, 20) (3996,)
Test set shape: (1000, 20) (1000,)


In [86]:
#Standardisation 

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [87]:
#Random Forest model sans rééchantillonnage
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define initial parameter grid for RandomizedSearchCV
random_search_params = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(3, 20, 1),
    'min_samples_split': np.arange(2, 21, 2)
}

# Initialize the model
rf_model = RandomForestClassifier()

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=random_search_params, n_iter=50, cv=cv, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)

# Print the best parameters from RandomizedSearchCV
print("Best Parameters from RandomizedSearchCV:")
print(random_search.best_params_)
print("Best Score from RandomizedSearchCV:", random_search.best_score_)

# Define parameter grid for GridSearchCV based on RandomizedSearchCV results
grid_search_params = {
    'n_estimators': [random_search.best_params_['n_estimators'] - 50, random_search.best_params_['n_estimators'], random_search.best_params_['n_estimators'] + 50],
    'max_depth': [random_search.best_params_['max_depth'] - 1, random_search.best_params_['max_depth'], random_search.best_params_['max_depth'] + 1],
    'min_samples_split': [random_search.best_params_['min_samples_split'] - 2, random_search.best_params_['min_samples_split'], random_search.best_params_['min_samples_split'] + 2]
}

# Ensure no negative values in the grid search parameters
for param in grid_search_params:
    grid_search_params[param] = [max(2, x) for x in grid_search_params[param]]

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=grid_search_params, cv=cv, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best parameters from GridSearchCV
print("Best Parameters from GridSearchCV:")
print(grid_search.best_params_)
print("Best Score from GridSearchCV:", grid_search.best_score_)

# Train Final Model with Best Parameters from GridSearchCV
final_modelRF = RandomForestClassifier(n_estimators=grid_search.best_params_['n_estimators'],
                                       max_depth=grid_search.best_params_['max_depth'],
                                       min_samples_split=grid_search.best_params_['min_samples_split'])

# Fit the final model with the best parameters
final_modelRF.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelRF.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters from RandomizedSearchCV:
{'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 14}
Best Score from RandomizedSearchCV: 0.9269267834793492
Best Parameters from GridSearchCV:
{'max_depth': 13, 'min_samples_split': 4, 'n_estimators': 100}
Best Score from GridSearchCV: 0.9281786608260326
Testing Accuracy: 0.925

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       858
           1       0.95      0.50      0.65       142

    accuracy                           0.93      1000
   macro avg       0.93      0.75      0.81      1000
weighted avg       0.93      0.93      0.91      1000


Confusion Matrix:
[[854   4]
 [ 71  71]]


In [88]:
#Random Forest model avec rééchantillonnage
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_estimators': [100, 150, 200, 250],
              'max_depth': [3, 5, 7, 9, 11, 18],
              'min_samples_split': [2, 5, 10, 20]}  
rf_model = RandomForestClassifier()
clf = GridSearchCV(estimator=rf_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Random Forest:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelRF = RandomForestClassifier(n_estimators=clf.best_params_['n_estimators'],
                                       max_depth=clf.best_params_['max_depth'],
                                       min_samples_split=clf.best_params_['min_samples_split']) 

# Define resampling strategies
resampling_strategies = {
    "SMOTE": SMOTE(random_state=42),
    "Random Undersampling": RandomUnderSampler(random_state=42),
  
}

# Loop over each resampling strategy
for resampling_name, resampler in resampling_strategies.items():
    # Create a pipeline with resampling strategy and classifier
    pipeline = make_pipeline(resampler, final_modelRF)
    
    # Fit the pipeline
    pipeline.fit(x_train, y_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(x_test)
    
    # Evaluate performance
    print(f"Resampling Strategy: {resampling_name}")
    print(classification_report(y_test, y_pred))

# Fit the final model with the best parameters
final_modelRF.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelRF.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)
# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)


Best Parameters for Random Forest:
{'max_depth': 18, 'min_samples_split': 10, 'n_estimators': 200}
Best Score: 0.9274280350438048
Resampling Strategy: SMOTE
              precision    recall  f1-score   support

           0       0.94      0.97      0.96       858
           1       0.78      0.64      0.71       142

    accuracy                           0.92      1000
   macro avg       0.86      0.81      0.83      1000
weighted avg       0.92      0.92      0.92      1000

Resampling Strategy: Random Undersampling
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       858
           1       0.48      0.72      0.58       142

    accuracy                           0.85      1000
   macro avg       0.72      0.80      0.74      1000
weighted avg       0.88      0.85      0.86      1000

Testing Accuracy: 0.925

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96 

In [89]:
#Random Forest model
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [90]:
#Random Forest model
# Compute precision-recall curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
precision, recall, _ = precision_recall_curve(y_test, final_modelRF.predict_proba(x_test)[:, 1])
pr_auc = auc(recall, precision)

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.2f})', color='#AEC6CF')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()



In [91]:
#Random Forest model
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelRF, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()


In [92]:
#LogisticRegression model
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
logistic_model = LogisticRegression()
clf = GridSearchCV(estimator=logistic_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Logistic Regression:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelLR = LogisticRegression(C=clf.best_params_['C'])
final_modelLR.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelLR.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Best Parameters for Logistic Regression:
{'C': 0.1}
Best Score: 0.8648642052565707
Testing Accuracy: 0.866

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       858
           1       0.64      0.13      0.21       142

    accuracy                           0.87      1000
   macro avg       0.76      0.56      0.57      1000
weighted avg       0.84      0.87      0.83      1000

AUC: 0.557552775862635

Confusion Matrix:
[[848  10]
 [124  18]]


In [93]:
#LogisticRegression model
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [94]:
#LogisticRegression model
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelLR, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_19404\1305074962.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))


In [95]:
#XGBoost model sans rééchantillonnage
import pickle
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import numpy as np

# Define cross-validation strategy
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Define initial parameter grid for RandomizedSearchCV
random_search_params = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(3, 10, 1),
    'colsample_bylevel': [0.4, 0.6, 0.8, 1.0],
    'subsample': [0.4, 0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'colsample_bytree': [0.4, 0.6, 0.8, 1.0]
}

# Initialize the model
xgb_model = XGBClassifier()

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=random_search_params, n_iter=50, cv=cv, random_state=42, n_jobs=-1)
random_search.fit(x_train, y_train)

# Print the best parameters from RandomizedSearchCV
print("Best Parameters from RandomizedSearchCV:")
print(random_search.best_params_)
print("Best Score from RandomizedSearchCV:", random_search.best_score_)

# Define parameter grid for GridSearchCV based on RandomizedSearchCV results
grid_search_params = {
    'n_estimators': [random_search.best_params_['n_estimators'] - 50, random_search.best_params_['n_estimators'], random_search.best_params_['n_estimators'] + 50],
    'max_depth': [random_search.best_params_['max_depth'] - 1, random_search.best_params_['max_depth'], random_search.best_params_['max_depth'] + 1],
    'colsample_bylevel': [random_search.best_params_['colsample_bylevel'] - 0.1, random_search.best_params_['colsample_bylevel'], random_search.best_params_['colsample_bylevel'] + 0.1],
    'subsample': [random_search.best_params_['subsample'] - 0.1, random_search.best_params_['subsample'], random_search.best_params_['subsample'] + 0.1],
    'gamma': [random_search.best_params_['gamma'] - 0.05, random_search.best_params_['gamma'], random_search.best_params_['gamma'] + 0.05],
    'colsample_bytree': [random_search.best_params_['colsample_bytree'] - 0.1, random_search.best_params_['colsample_bytree'], random_search.best_params_['colsample_bytree'] + 0.1]
}

# Ensure no negative values in the grid search parameters
for param in grid_search_params:
    grid_search_params[param] = [max(0, x) for x in grid_search_params[param]]

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=grid_search_params, cv=cv, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best parameters from GridSearchCV
print("Best Parameters from GridSearchCV:")
print(grid_search.best_params_)
print("Best Score from GridSearchCV:", grid_search.best_score_)

# Train Final Model with Best Parameters from GridSearchCV
final_modelxgb = XGBClassifier(n_estimators=grid_search.best_params_['n_estimators'], 
                                max_depth=grid_search.best_params_['max_depth'],
                                colsample_bylevel=grid_search.best_params_['colsample_bylevel'],
                                subsample=grid_search.best_params_['subsample'],
                                gamma=grid_search.best_params_['gamma'],
                                colsample_bytree=grid_search.best_params_['colsample_bytree'])

# Fit the final model with the best parameters
final_modelxgb.fit(x_train, y_train)

# Save the trained model to a file
with open("final_modelxgb.pkl", "wb") as f:
    pickle.dump(final_modelxgb, f)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelxgb.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters from RandomizedSearchCV:
{'subsample': 1.0, 'n_estimators': 150, 'max_depth': 9, 'gamma': 0.2, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.6}
Best Score from RandomizedSearchCV: 0.9259249061326658


c:\Users\chino\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
2025 fits failed out of a total of 3645.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\chino\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\chino\anaconda3\Lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\chino\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1519, in fit
    self._Booster = train(
                    ^^^^^^
  File "c:\Users\ch

Best Parameters from GridSearchCV:
{'colsample_bylevel': 0.7, 'colsample_bytree': 0.9, 'gamma': 0.2, 'max_depth': 10, 'n_estimators': 150, 'subsample': 0.9}
Best Score from GridSearchCV: 0.9284283479349187
Testing Accuracy: 0.926

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       858
           1       0.89      0.55      0.68       142

    accuracy                           0.93      1000
   macro avg       0.91      0.77      0.82      1000
weighted avg       0.92      0.93      0.92      1000

AUC: 0.7688203814964379

Confusion Matrix:
[[848  10]
 [ 64  78]]


In [96]:
#xgboost model avec rééchantillonnage
import pickle
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 7],
    'colsample_bylevel': [0.6, 0.8, 1.0],
    'subsample': [0.6, 0.8, 1.0],  
    'gamma': [0, 0.1, 0.2], 
    'colsample_bytree': [0.6, 0.8, 1.0]  
}
xgb_model = XGBClassifier()
clf = GridSearchCV(estimator=xgb_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for XGBoost:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelxgb = XGBClassifier(n_estimators=clf.best_params_['n_estimators'], 
                                max_depth=clf.best_params_['max_depth'],
                                colsample_bylevel=clf.best_params_['colsample_bylevel'],
                                subsample=clf.best_params_['subsample'],
                                gamma=clf.best_params_['gamma'],
                                colsample_bytree=clf.best_params_['colsample_bytree'])

# Define resampling strategies
resampling_strategies = {
    "SMOTE": SMOTE(random_state=42),
    "Random Undersampling": RandomUnderSampler(random_state=42),
 
}

# Loop over each resampling strategy
for resampling_name, resampler in resampling_strategies.items():
    # Create a pipeline with resampling strategy and classifier
    pipeline = make_pipeline(resampler, final_modelxgb)
    
    # Fit the pipeline
    pipeline.fit(x_train, y_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(x_test)
    
    # Evaluate performance
    print(f"Resampling Strategy: {resampling_name}")
    print(classification_report(y_test, y_pred))

# Fit the final model with the best parameters
final_modelxgb.fit(x_train, y_train)


# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelxgb.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for XGBoost:
{'colsample_bylevel': 1.0, 'colsample_bytree': 0.8, 'gamma': 0.2, 'max_depth': 7, 'n_estimators': 100, 'subsample': 1.0}
Best Score: 0.9259249061326658
Resampling Strategy: SMOTE
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       858
           1       0.85      0.59      0.70       142

    accuracy                           0.93      1000
   macro avg       0.89      0.79      0.83      1000
weighted avg       0.92      0.93      0.92      1000

Resampling Strategy: Random Undersampling
              precision    recall  f1-score   support

           0       0.95      0.81      0.88       858
           1       0.40      0.75      0.52       142

    accuracy                           0.80      1000
   macro avg       0.67      0.78      0.70      1000
weighted avg       0.87      0.80      0.83      1000

Testing Accuracy: 0.931

Classification Report:
              precision    recall  f1-score   su

In [97]:
#xgboost model
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [98]:
#xgboost model
# Compute ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, final_modelxgb.predict_proba(x_test)[:, 1])

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve', color='b')
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [99]:
#xgboost model
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelxgb, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

In [100]:
#knn model
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan', 'chebyshev']  # Adding distance metrics
}
knn_model = KNeighborsClassifier()
clf = GridSearchCV(estimator=knn_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for K-Nearest Neighbors:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelknn = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'], 
                                      metric=clf.best_params_['metric'])
final_modelknn.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelknn.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for K-Nearest Neighbors:
{'metric': 'manhattan', 'n_neighbors': 5}
Best Score: 0.8848851689612015
Testing Accuracy: 0.889

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       858
           1       0.84      0.27      0.41       142

    accuracy                           0.89      1000
   macro avg       0.87      0.63      0.67      1000
weighted avg       0.88      0.89      0.86      1000

AUC: 0.6297235628221544

Confusion Matrix:
[[851   7]
 [104  38]]


In [101]:
#knn model
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='twilight', cbar=False,linewidths=1, linecolor='gray')
y_labels =['Non Churn(0)','Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [102]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_modelknn, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score',color='#FFB6C1')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Testing Score',color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

In [103]:
#Decision Tree model sans rééchantillonnage
from sklearn.tree import DecisionTreeClassifier

# Decision Tree model
# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {
    'max_depth': [3, 5, 7, 9, 11],
    'criterion': ['gini', 'entropy']  # Adding criterion parameter
}
dt_model = DecisionTreeClassifier()
clf = GridSearchCV(estimator=dt_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Decision Tree:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelDT = DecisionTreeClassifier(max_depth=clf.best_params_['max_depth'],
                                       criterion=clf.best_params_['criterion'])  # Adding criterion parameter
final_modelDT.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelDT.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for Decision Tree:
{'criterion': 'entropy', 'max_depth': 5}
Best Score: 0.9144152065081352
Testing Accuracy: 0.921

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       858
           1       0.87      0.52      0.65       142

    accuracy                           0.92      1000
   macro avg       0.90      0.75      0.80      1000
weighted avg       0.92      0.92      0.91      1000

AUC: 0.7541531238714337

Confusion Matrix:
[[847  11]
 [ 68  74]]


In [104]:
#Decision Tree model avec rééchantillonnage
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {
    'max_depth': [3, 5, 7, 9, 11],
    'criterion': ['gini', 'entropy']  # Adding criterion parameter
}
dt_model = DecisionTreeClassifier()
clf = GridSearchCV(estimator=dt_model, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Decision Tree:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_modelDT = DecisionTreeClassifier(max_depth=clf.best_params_['max_depth'],
                                       criterion=clf.best_params_['criterion'])  # Adding criterion parameter

# Define resampling strategies
resampling_strategies = {
    "Random Oversampling": RandomOverSampler(random_state=42),
    "SMOTE": SMOTE(random_state=42),
    "Random Undersampling": RandomUnderSampler(random_state=42)
}

# Loop over each resampling strategy
for resampling_name, resampler in resampling_strategies.items():
    # Create a pipeline with resampling strategy and classifier
    pipeline = make_pipeline(resampler, final_modelDT)
    
    # Fit the pipeline
    pipeline.fit(x_train, y_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(x_test)
    
    # Evaluate performance
    print(f"Resampling Strategy: {resampling_name}")
    print(classification_report(y_test, y_pred))

# Fit the final model with the best parameters
final_modelDT.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_modelDT.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for Decision Tree:
{'criterion': 'entropy', 'max_depth': 5}
Best Score: 0.9144152065081352
Resampling Strategy: Random Oversampling
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       858
           1       0.68      0.61      0.64       142

    accuracy                           0.90      1000
   macro avg       0.81      0.78      0.79      1000
weighted avg       0.90      0.90      0.90      1000

Resampling Strategy: SMOTE
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       858
           1       0.61      0.62      0.62       142

    accuracy                           0.89      1000
   macro avg       0.77      0.78      0.78      1000
weighted avg       0.89      0.89      0.89      1000

Resampling Strategy: Random Undersampling
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       858
           1       0

In [105]:
#Decision Tree model
# Compute confusion matrix
conf_matrix_DT = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_DT, annot=True, fmt='d', cmap='twilight', cbar=False, linewidths=1, linecolor='gray')
y_labels = ['Non Churn(0)', 'Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [106]:
#Decision Tree model
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes_dt, train_scores_dt, valid_scores_dt = learning_curve(final_modelDT, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes_dt, np.mean(train_scores_dt, axis=1), label='Training Score', color='#FFB6C1')
plt.plot(train_sizes_dt, np.mean(valid_scores_dt, axis=1), label='Testing Score', color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve - Decision Tree')
plt.legend(loc='best')
plt.show()


In [107]:
# Naive Bayes model
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix


# Initialize Naive Bayes classifier model
nb_model = GaussianNB()

# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'var_smoothing': [1e-9, 1e-8, 1e-7]}  # Grid search over var_smoothing parameter
clf = GridSearchCV(estimator=nb_model, param_grid=parameters, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Naive Bayes:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)

# Train Final Model with Best Parameters
final_model_nb = GaussianNB(var_smoothing=clf.best_params_['var_smoothing'])
final_model_nb.fit(x_train, y_train)

# Evaluate Final Model
# Make predictions on the test set
y_pred = final_model_nb.predict(x_test)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print("Testing Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Best Parameters for Naive Bayes:
{'var_smoothing': 1e-09}
Best Score: 0.8836351689612014
Testing Accuracy: 0.893

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       858
           1       0.65      0.54      0.59       142

    accuracy                           0.89      1000
   macro avg       0.79      0.74      0.76      1000
weighted avg       0.89      0.89      0.89      1000

AUC: 0.743712859910043

Confusion Matrix:
[[817  41]
 [ 66  76]]


In [108]:
# Naive Bayes model
# Compute confusion matrix
conf_matrix_NB = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_NB, annot=True, fmt='d', cmap='twilight', cbar=False, linewidths=1, linecolor='gray')
y_labels = ['Non Churn(0)', 'Churn(1)']
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')

plt.gca().set_xticklabels(y_labels)
plt.gca().set_yticklabels(y_labels)
plt.show()


In [109]:
# Naive Bayes model
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes_dt, train_scores_dt, valid_scores_dt = learning_curve(nb_model, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes_dt, np.mean(train_scores_dt, axis=1), label='Training Score', color='#FFB6C1')
plt.plot(train_sizes_dt, np.mean(valid_scores_dt, axis=1), label='Testing Score', color='#AEC6CF')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()


In [110]:
#AUC/ROC Curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier

# Define the models
models = {
    'Random Forest': rf_model,
    'KNN': knn_model,
    'Logistic Regression': logistic_model,
    'Decision Tree': dt_model,
    'Naive Bayes': nb_model,
    'XGBoost': xgb_model
}



# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for name, model in models.items():
    model.fit(x_train, y_train)  
    y_scores = model.predict_proba(x_test)
    fpr, tpr, _ = roc_curve(y_test, y_scores[:, 1])
    roc_auc = auc(fpr, tpr)  # Ensure auc is the function, not a variable
    if name == 'XGBoost':
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})', color='navy')  
    else:
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


In [111]:
#tester si on a :underfitting /overftting /bestfit
print("Le score de train est: ", round(final_modelRF.score(x_train,y_train),2))
print("Le score de test est: ", round(final_modelRF.score(x_test,y_test),2))

Le score de train est:  0.96
Le score de test est:  0.92


In [112]:
#Enregistrer le modele dans un fichier
import pickle
with open("final_modelRF.pkl", "wb") as f:
    pickle.dump(final_modelRF,f)

In [113]:
model = pickle.load(open("final_modelRF.pkl", "rb"))